In [3]:
#AI-TECHGYM-2-7-A-3
#特徴量エンジニアリング

#インポート
import pandas as pd
import requests,io
import category_encoders as ce

#自動車価格データの取得
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content
auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)
auto.columns =['symboling','normalized-losses','make','fuel-type' ,'aspiration','num-of-doors',
                            'body-style','drive-wheels','engine-location','wheel-base','length','width','height',
                            'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore',
                            'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']
#データ表示
#display(auto)

#===one-hot-encoding===
#Eoncodeしたい列をリストで指定(複数も指定可能)
list_cols = ['body-style','engine-type']
auto_c = auto[['body-style','engine-type']]

#OneHotEncodeしたい列を指定(Nullや不明の場合の補完方法も指定)
ce_ohe = ce.OneHotEncoder(cols=list_cols)

#変換後のデータフレーム
auto_ce = ce_ohe.fit_transform(auto_c)

#表示
display(auto_ce)


,body-style_1,body-style_2,body-style_3,body-style_4,body-style_5,engine-type_1,engine-type_2,engine-type_3,engine-type_4,engine-type_5,engine-type_6,engine-type_7
0,1,0,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0
2,0,1,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
200,0,0,1,0,0,0,0,1,0,0,0,0
201,0,0,1,0,0,0,0,1,0,0,0,0
202,0,0,1,0,0,0,1,0,0,0,0,0
203,0,0,1,0,0,0,0,1,0,0,0,0


In [22]:
auto

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
201,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
202,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
203,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470


In [16]:
auto['make']

0      alfa-romero
1      alfa-romero
2      alfa-romero
3             audi
4             audi
          ...     
200          volvo
201          volvo
202          volvo
203          volvo
204          volvo
Name: make, Length: 205, dtype: object

In [57]:
ls1 = [[2,3]]
print(ls1)
#new_df
display(pd.DataFrame(columns=["row", "vector"]))

#tmp_se
display(pd.DataFrame(columns=["row", "vector"]).append(pd.Series([2,3],index=new_df.columns), ignore_index=True))

[[2, 3]]


,row,vector


,row,vector
0,2,3


In [53]:
ls1 = pd.Series([2,3],index=new_df.columns)
ls1

row       2
vector    3
dtype: int64

In [6]:
df_c

,0_Java,0_JavaScript,0_PHP,0_Python,0_Ruby
0,0,0,0,1,0
1,0,0,0,0,1
2,0,0,1,0,0
3,1,0,0,0,0
4,0,1,0,0,0


In [7]:
df_d

,0_Python,0_Ruby,0_PHP,0_Java,0_JavaScript
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,0,1


In [10]:
values

array(['Python', 'Ruby', 'PHP', 'Java', 'JavaScript'], dtype=object)